In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext import datasets
from torchtext import data
from torchtext.data import Field, LabelField
from torchtext.data import BucketIterator
import numpy as np
from tqdm import tqdm

#Prepare model

In [91]:
class GRU_Text(nn.Module):
    def __init__(self,
                 vocab_size,
                 emb_dim,
                 hidden_dim,
                 out_dim,
                 dropout,
                 pad_idx
                 ):
        super().__init__()
        self.embedding = nn.Embedding(
            vocab_size,
            emb_dim,
            padding_idx=pad_idx
            )
        self.rnn = nn.GRU(
            emb_dim,
            hidden_dim,
            num_layers=1,
            bidirectional=True
            )
        self.fc = nn.Linear(hidden_dim * 2, out_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        #text = [sent len, batch size]
        embedded = self.dropout(self.embedding(text))
        #embedded = [sent len, batch size, emb dim]
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.detach().cpu())
        packed_output, hidden = self.rnn(packed_embedded)
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        #hidden = [batch size, hid dim * num directions]
        return self.fc(hidden)

# Prepare dataset


In [92]:
torch.backends.cudnn.deterministic = True
TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)
train_data, test_data = datasets.TREC.splits(TEXT, LABEL)
train_data, valid_data = train_data.split()
TEXT.build_vocab(train_data, max_size=25000, vectors='glove.6B.100d', unk_init=torch.Tensor.normal_)
LABEL.build_vocab(train_data)
BATCH_SIZE = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

Add pretrained embeddings to increase quality

In [93]:
model = GRU_Text(
    vocab_size=len(TEXT.vocab),
    emb_dim=100,
    hidden_dim=128,
    out_dim=6,
    dropout=0.1,
    pad_idx=TEXT.vocab.stoi[TEXT.pad_token]
    )

In [94]:
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.5494, -1.0460,  0.4548,  ...,  1.5497,  0.6393, -1.4256],
        [-0.0999,  0.1005, -1.6691,  ...,  1.3173, -0.3409, -0.1412],
        [ 0.1638,  0.6046,  1.0789,  ..., -0.3140,  0.1844,  0.3624],
        ...,
        [ 0.0091,  0.2810,  0.7356,  ..., -0.7508,  0.8967, -0.7631],
        [ 0.2906,  0.3217,  0.2419,  ..., -0.9444, -0.3790,  0.6196],
        [-0.2803,  0.3769, -1.3597,  ..., -0.5477,  0.3796,  1.9748]])

In [95]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(100)
model.embedding.weight.data[TEXT.vocab.stoi[TEXT.pad_token]] = torch.zeros(100)

In [96]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [97]:
criterion = nn.CrossEntropyLoss()
model = model.to(device)
criterion = criterion.to(device)

In [98]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim=1, keepdim=True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    correct = correct.detach().to('cpu')
    return correct.sum() / torch.FloatTensor([y.shape[0]])

In [99]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    for batch in iterator:
        optimizer.zero_grad()
        text, text_lengths = batch.text
        predictions = model(text, text_lengths)
        loss = criterion(predictions, batch.label.long())
        acc = categorical_accuracy(predictions, batch.label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [100]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            text, text_lengths = batch.text
            predictions = model(text, text_lengths)
            loss = criterion(predictions, batch.label.long())
            acc = categorical_accuracy(predictions, batch.label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [101]:
import time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [102]:
N_EPOCHS = 50
best_valid_loss = float('inf')
for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'textrnn_trec.pt')
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 0s
	Train Loss: 1.251 | Train Acc: 50.20%
	 Val. Loss: 0.966 |  Val. Acc: 63.03%
Epoch: 02 | Epoch Time: 0m 0s
	Train Loss: 0.792 | Train Acc: 71.71%
	 Val. Loss: 0.689 |  Val. Acc: 74.79%
Epoch: 03 | Epoch Time: 0m 0s
	Train Loss: 0.578 | Train Acc: 79.70%
	 Val. Loss: 0.560 |  Val. Acc: 78.65%
Epoch: 04 | Epoch Time: 0m 0s
	Train Loss: 0.420 | Train Acc: 84.92%
	 Val. Loss: 0.491 |  Val. Acc: 82.05%
Epoch: 05 | Epoch Time: 0m 0s
	Train Loss: 0.311 | Train Acc: 89.81%
	 Val. Loss: 0.456 |  Val. Acc: 83.73%
Epoch: 06 | Epoch Time: 0m 0s
	Train Loss: 0.215 | Train Acc: 93.27%
	 Val. Loss: 0.423 |  Val. Acc: 84.95%
Epoch: 07 | Epoch Time: 0m 0s
	Train Loss: 0.156 | Train Acc: 95.16%
	 Val. Loss: 0.427 |  Val. Acc: 84.83%
Epoch: 08 | Epoch Time: 0m 0s
	Train Loss: 0.101 | Train Acc: 97.11%
	 Val. Loss: 0.438 |  Val. Acc: 85.38%
Epoch: 09 | Epoch Time: 0m 0s
	Train Loss: 0.072 | Train Acc: 98.10%
	 Val. Loss: 0.487 |  Val. Acc: 85.60%
Epoch: 10 | Epoch Time: 0m 0